In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Extract and Save Data

In [3]:
def get_behaviors_list(exp_dir):
    """
    Finds first JAABA experiment and returns list of all behavioral files.
    """
    flybowl_path = os.path.join(exp_dir, next(os.walk(exp_dir))[1][0])
    return list(filter(lambda x: x.startswith('scores'), os.listdir(flybowl_path)))

def create_column_names(behaviors):
    """
    Creates a list of column names for the dataframe based on the behavioral files.
    This allows for any combination of behavioral files to be used.
    """
    column_names = ['Name']
    for behavior in behaviors:
        column_names += [behavior[7:-4] + ' - Total Occurrences', behavior[7:-4] + ' - Total Time (s)']
    return column_names

def get_a_flybowl_behavior_data(data):
    t0 = data['allScores'][0][0][5][0]
    t1 = data['allScores'][0][0][6][0]
    diff = np.subtract(t1, t0)

    total_occurrences, total_frames = 0, 0
    for fly in diff:
        total_occurrences += fly.shape[1]
        total_frames += np.sum(fly)

    return total_occurrences, total_frames/30

def get_all_behavioral_data(flybowl_dir, flybowl, behaviors):
    flybowl_row = [flybowl]
    for behavior in behaviors:
        flybowl_row += get_a_flybowl_behavior_data(sp.io.loadmat(os.path.join(flybowl_dir, behavior)))
    return flybowl_row

def get_collated_data(exp_dir, behaviors):
    df = pd.DataFrame(columns=create_column_names(behaviors))
    for flybowl in sorted(os.listdir(exp_dir)):
          print(flybowl)
          df.loc[len(df)] = get_all_behavioral_data(os.path.join(exp_dir, flybowl), flybowl, behaviors)
    return df

In [ ]:
dir_name = '/content/drive/MyDrive/0 Flybowl/48 hrs-JAABAData'
# dir_name = '/content/drive/MyDrive/0 Flybowl/2025 FlyBowl JABBA data'
folders = ['13.17.23', '13.25.49', '13.33.55', '13.42.27']
# folders = ['2-5-2025-30207JAABAData', '2-12-2025-30207JAABAData']  # 30207
# folders = ['2-20-2025-102864JAABAData', '3-4-2025-102864JAABAData', '3-26-2025-102864JAABAData']  # 102864

summary = pd.DataFrame([])
for folder in folders:
  path = os.path.join(dir_name, folder)

  behaviors_list = get_behaviors_list(path)
  behaviors_list.sort()

  temp = get_collated_data(path, behaviors_list).sort_values('Name')
  summary = pd.concat([summary, temp], ignore_index=True)

# this summary['group'] part is where groups are determined
summary['group'] = summary['Name']  # I just leave it as Q1, ..., Q4
# summary['group'] = summary['Name'].str.rsplit(' ').str[-1].str[1:2]  # original
summary.to_excel(os.path.join('/content/drive/MyDrive/0 Flybowl/test.xlsx'), index=False)

# Bar Plots

In [143]:
def plot(df, title, group_colors, output_folder, top_ylim=None):
  df = df[df.columns.drop(list(df.filter(regex=' - Total Occurrences')))]
  df = df.drop(['Name'], axis=1)

  renamed_cols = []
  for column in df.columns:
    renamed_cols.append(column[:column.find('-')])
  df.columns=renamed_cols

  t = 0.666
  x = np.arange(len(df.columns)-1)
  fig, ax = plt.subplots(figsize=(16, 10))

  groups = df['grou'].unique()
  w = 0.75/len(groups)
  s_w = 0.2/len(groups)

  means = df.groupby('grou').mean()

  bars = []
  for i in x:
    for j, group in enumerate(means.index):
      bars.append(ax.bar(i+j*w, means.loc[group, df.columns[i]], w, alpha=t, color=group_colors[group]))

  for i in x:
    for j, group in enumerate(means.index):
      grouping = df[df['grou']==group].iloc[:, i]
      ax.scatter(i+j*w+np.random.uniform(-1*s_w, s_w, len(grouping)), grouping, alpha=t, color=group_colors[group])

  ax.legend(bars, means.index, loc='upper left', ncol=len(means.index))
  ax.set_xticks(x+w*(1 if len(groups)%2 else 1.5), df.columns[:-1])
  ax.set_ylim(top=top_ylim)
  ax.set_ylabel('Total Time (s)')
  ax.set_title(title)
  ax.spines[['right', 'top']].set_visible(False)

  fig.tight_layout()
  fig.patch.set_alpha(0)
  ax.patch.set_alpha(0)
  fig.savefig(os.path.join(output_folder, f'{title}_{top_ylim}.pdf'), transparent=True)

In [ ]:
# you can use named colors (https://matplotlib.org/stable/gallery/color/named_colors.html) or hexadecimal
colors = {'Q1':'tab:blue', 'Q2':'tab:orange', 'Q3':'tab:green', 'Q4':'tab:red'}
# colors = {1:'tab:blue', 2:'tab:orange', 3:'tab:green'}  # original
output_folder = '/content/drive/MyDrive/0 Flybowl'
plot(pd.read_excel('/content/drive/MyDrive/0 Flybowl/test.xlsx'), 'test', colors, output_folder, top_ylim=None)